# Exporting Data

In [1]:
import os.path
import glob
import gc
import pandas as pd
import joblib
import DER

In [2]:
simulations = {}
for i in glob.glob("results/*"):
    name = os.path.basename(i).split('.')[0]
    if name == "r_once_reb_26(PV)_sun_4"or name == "r_reg_reb_26(PV)_sun_4":
        simulations[name] = joblib.load(i)
        simulations[name].title = os.path.basename(i).replace(".gz", "").replace("_", " ")
        print(name)

r_once_reb_26(PV)_sun_4
r_reg_reb_26(PV)_sun_4


In [3]:
list(simulations.values())[0].lses[0].demand

array([7.53076845, 7.30644237, 7.15758658, ..., 5.49153616, 5.10787331,
       4.76642872], shape=(271752,))

In [4]:
# Time Series for Every LSE
tmp = pd.concat(
    [
        pd.DataFrame(
            {
                ("Active Number of Customers (Monthly Min)", s.title, lse.name): 
                lse.number_of_active_customers
                for s in simulations.values()
                for lse in s.lses
                if 'Residential' in lse.name
            },
            index = list(simulations.values())[0].time
        ).resample('m').min(),
        pd.DataFrame(
            {
                ("LMPs (Monthly Mean)", s.title, lse.name): 
                lse.lmp
                for s in simulations.values()
                for lse in s.lses
                if 'Residential' in lse.name
            },
            index = list(simulations.values())[0].time
        ).resample('m').mean(),
        pd.DataFrame(
            {
                ("Ramsey Price (Monthly Mean)", s.title, lse.name): 
                lse.ramsey_price
                for s in simulations.values()
                for lse in s.lses
                if 'Residential' in lse.name
            },
            index = list(simulations.values())[0].time
        ).resample('m').mean(),
        pd.DataFrame(
            {
                ("Demand (Monthly Sum)", s.title, lse.name): 
                lse.demand
                for s in simulations.values()
                for lse in s.lses
                if 'Residential' in lse.name
            },
            index = list(simulations.values())[0].time
        ).resample('m').sum(),
        pd.DataFrame(
            {
                ("Bill [ramsey*demand/#customers] (Monthly Sum)", s.title, lse.name): 
                lse.demand * lse.ramsey_price / lse.number_of_active_customers
                for s in simulations.values()
                for lse in s.lses
                if 'Residential' in lse.name
            },
            index = list(simulations.values())[0].time
        ).resample('m').sum(),
        pd.DataFrame(
            {
                ("Bill [LMP*demand/#customers] (Monthly Sum)", s.title, lse.name): 
                lse.demand * lse.lmp / lse.number_of_active_customers
                for s in simulations.values()
                for lse in s.lses
                if 'Residential' in lse.name
            },
            index = list(simulations.values())[0].time
        ).resample('m').sum(),
    ],
    axis=1
)
tmp = tmp.stack().stack().reset_index()
tmp = tmp.rename(
    columns={"level_0":"Time","level_1":"LSE Name","level_2":"Simulation Name"}
)
tmp['High Income?'] = tmp['LSE Name'].str.contains("High_Income")
tmp['Dynamic Price (Ramsey)?'] = tmp['Simulation Name'].str.contains("r reg")
tmp.to_clipboard()
tmp

C:\Users\vtt513\AppData\Local\Temp\ipykernel_17908\2156148555.py:13: FutureWarning: 'm' is deprecated and will be removed in a future version, please use 'ME' instead.
  ).resample('m').min(),
C:\Users\vtt513\AppData\Local\Temp\ipykernel_17908\2156148555.py:23: FutureWarning: 'm' is deprecated and will be removed in a future version, please use 'ME' instead.
  ).resample('m').mean(),
C:\Users\vtt513\AppData\Local\Temp\ipykernel_17908\2156148555.py:33: FutureWarning: 'm' is deprecated and will be removed in a future version, please use 'ME' instead.
  ).resample('m').mean(),
C:\Users\vtt513\AppData\Local\Temp\ipykernel_17908\2156148555.py:43: FutureWarning: 'm' is deprecated and will be removed in a future version, please use 'ME' instead.
  ).resample('m').sum(),
C:\Users\vtt513\AppData\Local\Temp\ipykernel_17908\2156148555.py:53: FutureWarning: 'm' is deprecated and will be removed in a future version, please use 'ME' instead.
  ).resample('m').sum(),
C:\Users\vtt513\AppData\Local\Tem

,Time,LSE Name,Simulation Name,Active Number of Customers (Monthly Min),LMPs (Monthly Mean),Ramsey Price (Monthly Mean),Demand (Monthly Sum),Bill [ramsey*demand/#customers] (Monthly Sum),Bill [LMP*demand/#customers] (Monthly Sum),High Income?,Dynamic Price (Ramsey)?
0,2020-01-31,BOWLING_GREEN_Residential_High_Income,r once reb 26(PV) sun 4,1722.0,16.032657,NaN,6530.659532,0.000000,61.218853,True,False
1,2020-01-31,BOWLING_GREEN_Residential_High_Income,r reg reb 26(PV) sun 4,1722.0,16.032657,NaN,6530.659532,0.000000,61.218853,True,True
2,2020-01-31,BOWLING_GREEN_Residential_Low_Income,r once reb 26(PV) sun 4,20964.0,16.032657,NaN,79505.659947,0.000000,61.218853,False,False
3,2020-01-31,BOWLING_GREEN_Residential_Low_Income,r reg reb 26(PV) sun 4,20964.0,16.032657,NaN,79505.659947,0.000000,61.218853,False,True
4,2020-01-31,CHATTANOOGA_Residential_High_Income,r once reb 26(PV) sun 4,14229.0,25.565336,NaN,53963.272056,0.000000,110.485154,True,False
...,...,...,...,...,...,...,...,...,...,...,...
23803,2050-12-31,STARKVILLE_Residential_Low_Income,r reg reb 26(PV) sun 4,6050.0,14.309857,104.432901,22944.579255,396.060232,54.669758,False,True
23804,2050-12-31,TUPELO_Residential_High_Income,r once reb 26(PV) sun 4,142.0,20.067235,83.677007,538.537273,317.343812,79.103516,True,False
23805,2050-12-31,TUPELO_Residential_High_Income,r reg reb 26(PV) sun 4,142.0,19.862586,92.414421,538.537273,350.480326,78.281590,True,True
23806,2050-12-31,TUPELO_Residential_Low_Income,r once reb 26(PV) sun 4,6393.0,20.067235,83.677007,24245.401682,317.343812,79.103516,False,False
